# CNN transformer Model

Initialise code for google colab

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_addons as tfa
print(tfa.__version__)

In [ ]:
!pip install tensorflow==2.11
!pip install tensorflow-addons==0.19

Create data base files under google colab environment

In [ ]:
!unzip -q '/content/drive/MyDrive/data_equalize.zip' -d '/content/'

For Luc because my archive made on mac create a __MACOSX folder

In [ ]:
%rm -rf /content/__MACOSX

Define working directory to our jupyter repertory:
* because path to the different repertories (./data, ./output...) are define relatevly to jupyter one
* let import _mypath which add ./lib to python path in order to import our own define libraries


In [ ]:
%cd /content/drive/MyDrive/covid-19-xRay/jupyter

In [ ]:
import _mypath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
from database.path_origin_data import build_data_paths 
from database.path_origin_data import lung_name, infection_name
from database.path_origin_data import train_name, test_name, valid_name
from database.path_origin_data import normal_name, covid_name, no_covid_name
from database.path_origin_data import images_name, lung_mask_name, infection_mask_name

from database.dataset import build_dataset

Build paths and variables for reading data base hierarchy

In [ ]:
# for local use
db_path = '../data_equalize'


In [ ]:
# for google colab use
db_path = '/content/data_equalize'


In [ ]:
output_path = os.path.join('..', 'output')
if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)


Structure to manage paths in data base

In [ ]:
data_paths = build_data_paths()
idx = pd.IndexSlice


Create tf Dataset

In [ ]:

paths = data_paths['path']

ds_train = build_dataset(db_path, paths, db=[lung_name], ds=[train_name])
ds_test = build_dataset(db_path, paths, db=[lung_name], ds=[test_name])
ds_valid = build_dataset(db_path, paths, db=[lung_name], ds=[valid_name])
ds_train

Model parameters

In [ ]:
batch_size = 32
image_size = 256
input_shape = (image_size, image_size, 1)

learning_rate = 0.001
weight_decay = 0.0001
num_epochs = 100

label_smoothing = 0.01

# data augmentation
scale = 1. / 255.
flip = "horizontal"
rotation_factor = 0.1
zoom_height_factor = 0.2
zoom_width_factor = 0.2

# vit
patch_size = 16
input_image_size = image_size
transformer_layers = 2
num_heads = 4
projection_dim = 64
transformer_units_rate = [2, 1]
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier



Data augmentation

In [ ]:
data_augmentation_seq = keras.Sequential(
    [
      layers.Rescaling(scale=scale),
      layers.RandomFlip(flip),
      layers.RandomRotation(rotation_factor),
      layers.RandomZoom(height_factor=zoom_height_factor, width_factor=zoom_width_factor),
    ],
    name='data_augmentation'
)

cnn

In [ ]:
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout

# Initialising the CNN
convolution_seq = keras.Sequential(
    [
      layers.Conv2D(64, (3, 3), activation = 'relu', padding='same', kernel_initializer='random_normal'),
      layers.MaxPooling2D(pool_size = (2, 2)),
      layers.Conv2D(64, (5, 5), activation = 'relu', padding='same', kernel_initializer='random_normal')
    ],
    name='convolution_net'
)


In [ ]:
from myLayers.vision_transformer import add_vit
from myLayers.mlp import mlp

inputs = layers.Input(shape=input_shape)
augmented = data_augmentation_seq(inputs)
cnn = convolution_seq(augmented)
features = add_vit(cnn,
            patch_size=patch_size,
            input_image_size=cnn.shape[1],
            transformer_layers=transformer_layers,
            num_heads=num_heads,
            projection_dim=projection_dim,
            transformer_units_rate=transformer_units_rate,
            mlp_head_units=mlp_head_units)
# Classify outputs.
softmax = layers.Dense(3, activation='softmax', kernel_initializer='random_normal')(features)

# Create the Keras model.
model = keras.Model(inputs=inputs, outputs=softmax)


In [ ]:
model.summary()

In [ ]:
from run_exp.standard import run_experiment

checkpoint_filepath = os.path.join(output_path, "checkpoint_cnn_transformer")
run_experiment(model,
              ds_train, ds_test, ds_valid,
              batch_size=batch_size, num_epochs=num_epochs,
              learning_rate=learning_rate, weight_decay=weight_decay,
              checkpoint_filepath=checkpoint_filepath,
              from_logits=False, label_smoothing=label_smoothing)